# Experimenting with analysing survey data

- Load and clean up the data
- Visualise using embeddings
- Categorise using GPT-4

In [1]:
from dhp_options_survey_analysis import PROJECT_DIR

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

import umap
import hdbscan
import altair as alt

/opt/homebrew/Caskroom/miniconda/base/envs/dhp_options_survey_analysis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-11-08 22:41:05,016 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-11-08 22:41:05,383 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu


## Load the data

In [10]:
# Path to data
OUTPUTS_PATH = PROJECT_DIR / "outputs/data"
FILEPATH = PROJECT_DIR / "inputs/data/KK Copy of test data for DS on options issue map.xlsx"
SHEETS = [
    "g&p issues",
    "g&p interventions",
    "health issues",
    "health interventions",
    "inequality issues",
    "inequality interventions",
    "education issues",
    "education interventions",
]
# Define columns that we'll use
issue_cols = [f"Q_issue_{i}" for i in range(1, 11)]
other_cols = [f"q_other_{i}" for i in range(1, 12)]
intervention_cols = [f"q_intervention_{i}" for i in range(1, 11)]

In [11]:
def process_table(data_df: pd.DataFrame, sheet_name: str) -> pd.DataFrame:
    """Process on table of survey data"""
    if 'issues' in sheet_name:
        cols = issue_cols+other_cols
    else:
        cols = intervention_cols
    return (
        data_df
        .melt(value_vars=cols)
        .rename(columns={"variable": "question"})
        .assign(data_type=sheet_name)
        .assign(policy_area=lambda x: x.data_type.str.split(" ").str[0])
        .dropna(subset=['value'])
        .query("value != '-'")
    )

def load_and_process_survey() -> pd.DataFrame:
    """Load and process all survey data"""
    dfs = []
    for sheet_name in SHEETS:
        data_df = pd.read_excel(FILEPATH, sheet_name)
        dfs.append(process_table(data_df, sheet_name))
    return pd.concat(dfs, ignore_index=True)

In [12]:
survey_df = load_and_process_survey()

In [13]:
survey_df

,question,value,data_type,policy_area
0,Q_issue_1,Massively up-scaled and improved adult skills ...,g&p issues,g&p
1,Q_issue_1,Reduce energy prices as quickly as possible,g&p issues,g&p
2,Q_issue_1,"Increasing housing supply (private, not social)",g&p issues,g&p
3,Q_issue_1,Failure to build enough homes in the places th...,g&p issues,g&p
4,Q_issue_1,"Poor transport infrastructure, especially in t...",g&p issues,g&p
...,...,...,...,...
1666,q_intervention_10,Curriculum review,education interventions,education
1667,q_intervention_10,Roll out high quality universal early years pr...,education interventions,education
1668,q_intervention_10,give level 2 students access to funds similar ...,education interventions,education
1669,q_intervention_10,Urgently rebuild capacity in children's servic...,education interventions,education


## Embed and visualise

In [14]:
vectors = model.encode(survey_df.value.tolist(), show_progress_bar=True)

Batches: 100%|██████████| 53/53 [00:02<00:00, 21.41it/s]


In [18]:
# use umap to reduce dimensionality
umap_embeddings = umap.UMAP(
    n_neighbors=15,
    n_components=25,
).fit_transform(vectors)

In [19]:
# use hdbscan to cluster, and assing all points to a cluster
cluster = hdbscan.HDBSCAN(
    min_cluster_size=10,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True,
).fit(umap_embeddings)

In [20]:
# use umap to reduce to 2-d
umap_embeddings_2d = umap.UMAP(
    n_neighbors=15,
    n_components=2,
).fit_transform(vectors)

In [21]:
# use altair to plot the clusters
survey_viz_df = (
    survey_df
    .assign(cluster=cluster.labels_)
    .assign(x=umap_embeddings_2d[:, 0])
    .assign(y=umap_embeddings_2d[:, 1])    
)

fig = (
    alt.Chart(survey_viz_df)
    .mark_circle()
    .encode(
        x='x',
        y='y',
        color='cluster:N',
        tooltip=['value', 'cluster'],
    )
    .interactive()
)

fig

alt.Chart(...)

In [22]:
# use tf-idf to get top words from each cluster
# first, join up all values in each cluster
cluster_df = (
    survey_viz_df
    .groupby('cluster')
    .agg({'value': ' '.join})
    .reset_index()
)

In [23]:
# # then, get top words for each cluster
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(cluster_df.value.tolist())

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Create a dictionary to hold top words for each cluster
top_words_per_cluster = defaultdict(list)

# Number of top words you want to display per cluster
n_top_words = 5

# Iterate over each cluster and get top words
for cluster_idx, tfidf_scores in enumerate(tfidf_matrix):
    # Get indices of top n words within the cluster
    top_word_indices = tfidf_scores.toarray()[0].argsort()[:-n_top_words - 1:-1]
    
    # Get the top words corresponding to the top indices
    top_words = [feature_names[i] for i in top_word_indices]
    
    # Append the words to the dictionary
    top_words_per_cluster[cluster_df.iloc[cluster_idx]['cluster']] = top_words

# Print the top words for each cluster
cluster_names = [f"Cluster {cluster}: {', '.join(words)}" for cluster, words in top_words_per_cluster.items()]



In [24]:
survey_clustered_df = (
    survey_viz_df
    .assign(cluster_name=lambda x: x.cluster.map(dict(zip(top_words_per_cluster.keys(), cluster_names))))
)

In [25]:
survey_clustered_df.to_csv(OUTPUTS_PATH / "survey_clustered.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data'

In [ ]:
(
    survey_clustered_df
    .groupby(['cluster_name', 'policy_area'])
    .agg({'value': 'count'})
    .reset_index()
).to_csv("data/survey_clustered_counts.csv", index=False)